Il faudrait Donner la possibilité à l'utilisateur, soit de traiter tous les fichiers qui sont dans le current folder, soit de traiter spécifiquement un seul fichier.

Il faudrait aussi verifier si le fichier .json existe déjà, pour savoir si on fait une update ou si les data ont déjà été récupérées

#### Import packages

In [4]:
# Image processing
from PIL import Image 
import json

# for working on excel file
import xlrd  

# To get a list of the file in a folder
from glob import glob  

import numpy as np
import time

# Define Image size openable with PIL (default size is too small)
Image.MAX_IMAGE_PIXELS = 10000000000


In [8]:
def extractAnnotatedCentriole(xls, img, cropSize = 32)
    """
    INPUT: 
        Excel file with centriole coordinates
        Image corresponding to the excel
        Size of the image crop for each centriole
        Do you want to save
        
    OUTPUT:
        A dictionnary with coordinates, angle and image for each centrioleId
        A list with centriole unable to be extracted
        
    """
    # In the Excel file version XXX, the coordinate of centriole starts at row 7
    centrioleCount = 0
    centrioleDataset = {}   
    centrioleIssue = []

    # Iterate trough each row of the xls
    for row in range(xls.nrows - 7):

        X_Coord = xls_sheet.cell_value(row + 7 , 3)
        Y_Coord = xls_sheet.cell_value(row + 7 , 4)
        angle = xls_sheet.cell_value(row + 7, 1)
        centrioleCount += 1

        # Extract the centriole on the image
        if isinstance(X_coord, (float, int)) and isinstance(Y_coord, (float, int)):
            X_to_crop = X_Coord - size_img//2
            Y_to_crop = Y_Coord - size_img//2
            centriole_extracted = tif_file.crop((X_to_crop, Y_to_crop
                                                 , X_to_crop + size_img, Y_to_crop + size_img))

            # Try to crop the centriole into Numpy array
            try:
                centriole_extracted = np.asarray(centriole_extracted, dtype = "uint16")

                centriole = {'centrioleId': count_centriole
                             'X' : X_Coord,
                             'Y' : Y_Coord, 
                             'angle' : angle,
                             'image' : centriole_extracted.tolist()
                            }  

            except:
                centriole = {'issue' : 'nan in image',
                             'centrioleId': count_centriole,
                             'X' : X_Coord,
                             'Y' : Y_Coord, 
                             'angle' : angle
                             'image': 'issue'}


            # Check for issue
            if centriole['issue'] == 'nan in image':
                centrioleIssue.append(centriole)

            elif not isinstance(count_centriole, (float, int)):
                centrioleIssue.append(centriole['issue'] = 'centrioleId')

            elif not isinstance(X_Coord, (float, int)) or not isinstance(Y_Coord, (float, int)):
                centrioleIssue.append(centriole['issue'] = 'coordinates')

            elif not isinstance(angle, (float, int)):
                centrioleIssue.append(centriole['issue'] = 'angle')

            elif centriole_extracted.max() == 0:
                centrioleIssue.append(centriole['issue'] = 'black image')

            # If no issue try to add the centriole to the dictionnary
            else:   
                try: 
                    centrioleDataset[count_centriole] = {"X" : X_Coord,
                                                         "Y" : Y_Coord, 
                                                         "angle" : angle,
                                                         "image" : centriole_extracted.tolist()
                                                        }  
                except:
                    print('trouble in the creation of centriole coordinate')
                    
        else:
            # No centriole in the xls row
            pass

    return centrioleDataset, centrioleIssue

Extraction over


In [1]:
issueDatabase = []

# List all .xlsm files and .tif file in 'Analyzed' folder
xls_list = glob("./Analyzed/*.xlsm")
tif_list = glob("./Analyzed/*.tif")

# Test if there is as many tif as xls
if len(xls_list) != len(tif_list):
    raise Exception("A xls/tif file is missing: find the missing file or remove the extra file" )

# Test if each any xls fit with a tif
for i in range(max(len(xls_list), len(tif_list))):
    if xls_list[i].replace('.xlsm', '') != tif_list[i].replace('.tif', ''):
        raise Exception(f'{xls_list[i]} or {tif_list[i]} is not appariate with a tif/xsl'))
        
del tif_list

# Iteration trough the excel file
for xlsFile in xls_list:
    tif_name = xls_name.replace('.xlsm', '.tif')
    file_name = xls_name[xls_name.rfind('/')+1:].replace('.xlsm', '')
    
    # Open the image
    with Image.open(tif_name) as img:
        # Open the excel file 
        with xlrd.open_workbook(xlsFile) as xls:
            # Go to appropriate sheet (index 0)
            xls = xls.sheet_by_index(0)
            
            centrioleDatabse, issueList = extractAnnotatedCentriole( xls
                                                              , img
                                                              , centrioleIssue = None
                                                              , cropSize = 32 ) 
            
    issueDatabase.append(issueList)
    
    # Save each file
    with open((file_name + '.json'), 'w') as output: 
        json.dump(centrioleDatabase, output)
        
    with open('centrioleExtractionIssue.json', 'w') as output: 
        json.dump(issueDatabase, output)
       
print('extraction over')